## 📓 02_llm_grunnleggende.ipynb

# 🤖 Store Språkmodeller (LLM) - Grunnleggende konsepter

## Læringsmål
- Forstå hvordan LLM genererer tekst
- Lære om tokens og embeddings
- Utforske temperature og sampling

In [5]:
import tiktoken
from transformers import AutoTokenizer
import numpy as np

print("🚀 LLM Grunnleggende - Fra tekst til AI-forståelse")

🚀 LLM Grunnleggende - Fra tekst til AI-forståelse


## Tokenisering: Hvordan AI leser tekst

In [6]:
# Bruk OpenAI's tokenizer
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Medisinsk eksempel
tekst = "Pasienten har diabetes mellitus type 2 og hypertensjon."
tokens = encoding.encode(tekst)
token_strings = [encoding.decode([token]) for token in tokens]

print(f"Original tekst: {tekst}")
print(f"Antall tokens: {len(tokens)}")
print(f"Tokens: {token_strings}")

# Visualiser tokenisering
for i, (token, string) in enumerate(zip(tokens, token_strings)):
    print(f"Token {i}: '{string}' (ID: {token})")

Original tekst: Pasienten har diabetes mellitus type 2 og hypertensjon.
Antall tokens: 15
Tokens: ['Pas', 'ient', 'en', ' har', ' diabetes', ' mell', 'itus', ' type', ' ', '2', ' og', ' hypert', 'ens', 'jon', '.']
Token 0: 'Pas' (ID: 72011)
Token 1: 'ient' (ID: 1188)
Token 2: 'en' (ID: 268)
Token 3: ' har' (ID: 4960)
Token 4: ' diabetes' (ID: 20335)
Token 5: ' mell' (ID: 54448)
Token 6: 'itus' (ID: 36891)
Token 7: ' type' (ID: 955)
Token 8: ' ' (ID: 220)
Token 9: '2' (ID: 17)
Token 10: ' og' (ID: 7500)
Token 11: ' hypert' (ID: 48855)
Token 12: 'ens' (ID: 729)
Token 13: 'jon' (ID: 35265)
Token 14: '.' (ID: 13)


## Temperature: Kontrollere kreativitet vs presisjon

In [10]:
import numpy as np

def simulate_generation(probs, temperature=1.0):
    """
    Simuler hvordan temperature påvirker tekstgenerering
    """
    # Konverter til NumPy array hvis nødvendig
    probs = np.array(probs)
    
    # Juster sannsynligheter basert på temperature
    if temperature == 0:
        # Deterministisk: velg mest sannsynlige
        return np.argmax(probs)
    
    # Skaler log-probs med temperature
    log_probs = np.log(probs + 1e-10) / temperature
    # Konverter tilbake til sannsynligheter
    scaled_probs = np.exp(log_probs)
    scaled_probs = scaled_probs / np.sum(scaled_probs)
    
    # Sample fra distribusjonen
    return np.random.choice(len(probs), p=scaled_probs)

# Eksempel: Neste ord etter "Pasienten har"
mulige_ord = ["diabetes", "smerter", "feber", "hodepine", "kreft"]
sannsynligheter = [0.3, 0.25, 0.2, 0.15, 0.1]

print("🤖 Generering med ulike temperature-verdier:")
print("-" * 50)

# Sett seed for reproduserbare resultater
np.random.seed(42)

for temp in [0.0, 0.5, 1.0, 2.0]:
    valgte_ord = []
    for _ in range(5):
        idx = simulate_generation(sannsynligheter, temp)
        valgte_ord.append(mulige_ord[idx])
    
    print(f"Temperature {temp:3.1f}: {', '.join(valgte_ord)}")
    
    # Legg til forklaring
    if temp == 0.0:
        print("               ↳ Deterministisk - velger alltid 'diabetes' (høyest sannsynlighet)")
    elif temp == 0.5:
        print("               ↳ Konservativ - favoriserer sannsynlige ord")
    elif temp == 1.0:
        print("               ↳ Balansert - følger opprinnelig distribusjon")
    elif temp == 2.0:
        print("               ↳ Kreativ - mer tilfeldige valg")
    print()

# Vis hvordan temperature påvirker distribusjonen
print("📊 Hvordan temperature endrer sannsynlighetsfordelingen:")
print("-" * 50)

probs_array = np.array(sannsynligheter)
print("Opprinnelige sannsynligheter:")
for i, (ord, prob) in enumerate(zip(mulige_ord, sannsynligheter)):
    print(f"  {ord:<10}: {prob:.2f}")

print("\nMed temperature = 0.5 (mer fokusert):")
temp = 0.5
log_probs = np.log(probs_array + 1e-10) / temp
scaled_probs = np.exp(log_probs)
scaled_probs = scaled_probs / np.sum(scaled_probs)
for i, (ord, prob) in enumerate(zip(mulige_ord, scaled_probs)):
    print(f"  {ord:<10}: {prob:.2f}")

print("\nMed temperature = 2.0 (mer spredt):")
temp = 2.0
log_probs = np.log(probs_array + 1e-10) / temp
scaled_probs = np.exp(log_probs)
scaled_probs = scaled_probs / np.sum(scaled_probs)
for i, (ord, prob) in enumerate(zip(mulige_ord, scaled_probs)):
    print(f"  {ord:<10}: {prob:.2f}")  # Fixed the f-string formatting

🤖 Generering med ulike temperature-verdier:
--------------------------------------------------
Temperature 0.0: diabetes, diabetes, diabetes, diabetes, diabetes
               ↳ Deterministisk - velger alltid 'diabetes' (høyest sannsynlighet)

Temperature 0.5: diabetes, hodepine, feber, smerter, diabetes
               ↳ Konservativ - favoriserer sannsynlige ord

Temperature 1.0: diabetes, diabetes, hodepine, feber, feber
               ↳ Balansert - følger opprinnelig distribusjon

Temperature 2.0: diabetes, kreft, hodepine, diabetes, diabetes
               ↳ Kreativ - mer tilfeldige valg

📊 Hvordan temperature endrer sannsynlighetsfordelingen:
--------------------------------------------------
Opprinnelige sannsynligheter:
  diabetes  : 0.30
  smerter   : 0.25
  feber     : 0.20
  hodepine  : 0.15
  kreft     : 0.10

Med temperature = 0.5 (mer fokusert):
  diabetes  : 0.40
  smerter   : 0.28
  feber     : 0.18
  hodepine  : 0.10
  kreft     : 0.04

Med temperature = 2.0 (mer spredt)

In [11]:
# Importer nødvendige biblioteker
import tiktoken

# Hent encoding for GPT-modeller
encoding = tiktoken.get_encoding("cl100k_base")  # Brukes av GPT-3.5 og GPT-4

# Demonstrer kontekstvindu-begrensning
def estimate_tokens(text):
    """Estimer antall tokens i tekst"""
    return len(encoding.encode(text))

# Typiske medisinske dokumenter
dokumenter = {
    "Kort konsultasjon": 200,
    "Standard journalnotat": 500,
    "Omfattende sykehistorie": 2000,
    "Full pasientjournal": 10000,
    "Komplett pasientmappe": 50000,
    "Forskningsrapport": 25000
}

print("🏥 Token-estimat for ulike dokumenttyper:")
print("-" * 50)
for dok_type, tokens in dokumenter.items():
    print(f"{dok_type:<25}: ~{tokens:,} tokens")
    
    # Klassifiser basert på modell-kapasitet
    if tokens <= 4000:
        print(f"  ✅ Passer i GPT-3.5 (4K kontekst)")
    elif tokens <= 8000:
        print(f"  ⚠️  Trenger GPT-4 (8K) eller deling")
    elif tokens <= 32000:
        print(f"  🔄 Trenger GPT-4 Turbo (32K)")
    elif tokens <= 128000:
        print(f"  📚 Trenger GPT-4 Turbo (128K)")
    else:
        print(f"  ❌ Må deles opp eller sammendras")
    print()

# Praktisk eksempel med ekte tekst
eksempel_journalnotat = """
Pasient: 65 år gammel mann
Henvisning: Utredning av brystsmerter

Anamnese:
Pasienten har hatt tilbakevendende brystsmerter de siste 3 månedene.
Smertene er lokalisert substernalt, stråler til venstre arm.
Utløses ved anstrengelse, bedres ved hvile.
Ingen kjente hjertesykdommer i familien.
Røyker 20 sigaretter daglig i 40 år.

Status:
Allmenntilstand god, ikke tungpust i hvile.
BT: 150/95, Puls: 75/min regulær
Hjertelytting: Normale toner, ingen bilyder
Lunger: Normale respirasjonslyd

Supplerende undersøkelser:
EKG: Normalt sinusrytme, ingen ST-forandringer
Troponin: <0.01 (normalt)
Kolesterol: 6.8 mmol/L (høyt)

Vurdering:
Sannsynlig stabil angina pectoris
Kardiovaskulære risikofaktorer: Røyking, hypertensjon, hyperkolesterolemi

Plan:
1. Stress-EKG for å bekrefte diagnose
2. Ekkokardiografi
3. Livsstilsråd: røykeslutt, kostomlegging
4. Medisinering: ASA 75mg, statin
5. Kontroll om 3 måneder
"""

faktiske_tokens = estimate_tokens(eksempel_journalnotat)
print(f"📋 Eksempel journalnotat:")
print(f"Tekststørrelse: {len(eksempel_journalnotat)} tegn")
print(f"Estimerte tokens: {faktiske_tokens}")

if faktiske_tokens <= 4000:
    print("✅ Passer enkelt i alle moderne LLM-er")
else:
    print("⚠️ Kan være utfordrende for eldre modeller")

🏥 Token-estimat for ulike dokumenttyper:
--------------------------------------------------
Kort konsultasjon        : ~200 tokens
  ✅ Passer i GPT-3.5 (4K kontekst)

Standard journalnotat    : ~500 tokens
  ✅ Passer i GPT-3.5 (4K kontekst)

Omfattende sykehistorie  : ~2,000 tokens
  ✅ Passer i GPT-3.5 (4K kontekst)

Full pasientjournal      : ~10,000 tokens
  🔄 Trenger GPT-4 Turbo (32K)

Komplett pasientmappe    : ~50,000 tokens
  📚 Trenger GPT-4 Turbo (128K)

Forskningsrapport        : ~25,000 tokens
  🔄 Trenger GPT-4 Turbo (32K)

📋 Eksempel journalnotat:
Tekststørrelse: 908 tegn
Estimerte tokens: 341
✅ Passer enkelt i alle moderne LLM-er


## Kontekstvindu og begrensninger

LLMs har begrenset "hukommelse" (kontekstvindu):
- GPT-3.5: ~4,000 tokens
- GPT-4: 8,000-128,000 tokens
- Claude 3: 200,000 tokens

For medisinske journaler betyr dette at vi må:
1. Prioritere relevant informasjon
2. Dele opp lange dokumenter
3. Bruke sammendrag for historisk data

In [12]:
# Demonstrer kontekstvindu-begrensning
def estimate_tokens(text):
    """Estimer antall tokens i tekst"""
    return len(encoding.encode(text))

# Typiske medisinske dokumenter
dokumenter = {
    "Kort konsultasjon": 200,
    "Standard journalnotat": 500,
    "Omfattende sykehistorie": 2000,
    "Full pasientjournal": 10000
}

print("Token-estimat for ulike dokumenttyper:")
print("-" * 40)
for dok_type, tokens in dokumenter.items():
    print(f"{dok_type}: ~{tokens} tokens")
    if tokens <= 4000:
        print(f"  ✅ Passer i GPT-3.5")
    elif tokens <= 8000:
        print(f"  ⚠️  Trenger GPT-4 eller deling")
    else:
        print(f"  ❌ Må deles opp eller sammendras")

Token-estimat for ulike dokumenttyper:
----------------------------------------
Kort konsultasjon: ~200 tokens
  ✅ Passer i GPT-3.5
Standard journalnotat: ~500 tokens
  ✅ Passer i GPT-3.5
Omfattende sykehistorie: ~2000 tokens
  ✅ Passer i GPT-3.5
Full pasientjournal: ~10000 tokens
  ❌ Må deles opp eller sammendras


## 💭 Refleksjonsoppgave

1. Hvorfor er tokenisering viktig for medisinske termer?
2. Når bør vi bruke lav vs høy temperature i kliniske applikasjoner?
3. Hvordan kan vi håndtere lange pasientjournaler med begrenset kontekstvindu?